In [9]:
import os
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import torch
import torchvision.models as models
import torchvision.models.segmentation as segmentation
from datetime import date
import random
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torchmetrics.classification import MulticlassJaccardIndex as IoULoss
import sys
import numpy as np

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [3]:
class CustomDataset(Dataset):
    def __init__(self,root,img_dir, mask_dir, transform=None):
        self.root = root
        self.img_dir = os.path.join(root ,img_dir)
        self.mask_dir = os.path.join(root,mask_dir)
        
        self.transform = transform
        self.images = []
        self.masks = []
        # TODO delete it
        for file in os.listdir(img_dir):
            img_path = os.path.join(img_dir,file)
            mask_path = os.path.join(mask_dir,file.replace('jpg','png'))
            self.images.append(img_path)
            self.masks.append(mask_path)

    def __getitem__(self, index):
        img_path = self.images[index]
        mask_path = self.masks[index]
        image = Image.open(img_path).convert("RGB").resize((256,256))
        mask = Image.open(mask_path).convert("L").resize((256,256))
        mask = np.array(mask)
        if self.transform is not None:
            image = self.transform(image)
            mask = torch.from_numpy(mask)

        return {'image':image, 'mask':mask}

    def __len__(self):
        return len(self.images)


In [4]:
root = '.'
image_dir = '/kaggle/input/mapillary/Mapillary-Vistas-1000-sidewalks/training/images'
mask_dir = "/kaggle/input/mapillaryconvert/convert/training/labels"

val_img = "/kaggle/input/mapillary/Mapillary-Vistas-1000-sidewalks/validation/images"
val_mask = "/kaggle/input/mapillaryconvert/convert/validation/labels"
model_path = "/kaggle/input/deeplab-model-1/deeplab_model_final.pth"

In [5]:
dataset = CustomDataset(root,image_dir, mask_dir, transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
valdata = CustomDataset(root,val_img,val_mask,transform)
valloader = DataLoader(valdata,batch_size=32,shuffle=True)

In [6]:
deeplab_model = segmentation.deeplabv3_resnet50(pretrained=False, num_classes=8)
classifier = list(deeplab_model.classifier.children())
classifier.append(nn.Softmax(dim=1))
deeplab_model.classifier = nn.Sequential(*classifier)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 115MB/s] 


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
deeplab_model.load_state_dict(torch.load(model_path))

In [17]:
# avg=[0,0,0,0,0,0,0,0]
# for i in data_loader:
#     masks = torch.flatten(i['mask']).to(torch.long)
#     a=list(torch.bincount(masks).numpy())
#     for j in range(8):
#         avg[j]+=a[j]/sum(a)
weights = [12.701611518859863, 1.546370506286621, 3.8333635330200195, 0.06672064463297527, 0.03203535079956055, 0.22235329945882162, 0.5688924789428711, 0.028613567352294922]

In [19]:
weights = [1 / (8 * i) for i in weights]
weights = np.array(weights)/sum(weights)
#weights = np.array(avg)/len(data_loader)
loss_weights = torch.from_numpy(weights).float().to(device)

In [21]:
num_classes = 8
#batch_size = 5
learning_rate = 1e-4
num_epochs = 30
weight_decay = 1e-5

In [ ]:
criterion = nn.CrossEntropyLoss(weight=loss_weights)
optimizer = optim.Adam(deeplab_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
iou = IoULoss(num_classes=8).to(device)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
#nn.DataParallel(deeplab_model)
deeplab_model = deeplab_model.to(device)

In [ ]:
best_iou = float('inf')
# Training loop
for epoch in tqdm(range(num_epochs)):
    t_loss = 0.0
    deeplab_model.train()
    for batch in tqdm(data_loader):
        images = batch['image']
        masks = batch['mask']
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = deeplab_model(images)
        #masks = masks.squeeze(1)
        masks = masks.long()
        loss = criterion(outputs['out'], masks)
        t_loss += loss.item()
        loss.backward()
        optimizer.step()
        
    deeplab_model.eval()
    total_loss = 0.0
    total_iou = 0.0
    with torch.no_grad():
        for val_batch in tqdm(valloader):
            val_images = val_batch['image'].to(device)
            masks = val_batch['mask'].to(device)
            outputs = deeplab_model(val_images)
#             masks = masks.squeeze(1)
            masks = masks.long()
            #print(torch.argmax(outputs['out'],dim=1).size(),masks.size())
            v_loss = criterion(outputs['out'], masks)
            total_iou += float(iou(torch.argmax(outputs['out'], dim=1), masks))
            total_loss += v_loss.item()
        avg_val_loss = total_loss / len(valloader.dataset)
        avg_iou = total_iou/len(valloader.dataset)
        if epoch > 10:
            if avg_iou < best_iou :
                best_iou = avg_iou
                print(f'Saving the model at Epoch [{epoch + 1}/{num_epochs}]')
                torch.save(deeplab_model.state_dict(), 'deeplab_model_map.pth')
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {t_loss/len(data_loader.dataset)}, Val Loss: {avg_val_loss}, iou:{avg_iou}')